In [1]:
import torch
from torch import nn
from torch.nn import functional as f

In [17]:
a = torch.Tensor([[-2.3543e+00, -9.5574e+00,  3.2396e+01,  1.0000e-30,
          1.0000e-30,  1.0000e-30],
        [-5.7400e+00, -3.9120e+01, -1.2789e+01,   1.0000e-30,
          1.0000e-30,  1.0000e-30],
        [ 6.5512e+00,  1.3471e+01, -1.8644e+01,   1.0000e-30,
          1.0000e-30,  1.0000e-30]])

In [18]:
f.softmax(a,dim=1)

tensor([[ 8.0935e-16,  6.0238e-19,  1.0000e+00,  8.5231e-15,  8.5231e-15,
          8.5231e-15],
        [ 1.0704e-03,  3.4105e-18,  9.2944e-07,  3.3298e-01,  3.3298e-01,
          3.3298e-01],
        [ 9.8705e-04,  9.9901e-01,  1.1277e-14,  1.4099e-06,  1.4099e-06,
          1.4099e-06]])

In [3]:
a = [1,2,3]
b = [a]*3
b

[[1, 2, 3], [1, 2, 3], [1, 2, 3]]

In [4]:
b[0][0] = 5
b

[[5, 2, 3], [5, 2, 3], [5, 2, 3]]

In [7]:
hidden_size = 128
freqs = torch.Tensor(
            [10000 ** (-i / hidden_size) if i % 2 == 0 else -10000 ** ((1 - i) / hidden_size) for i in range(hidden_size)])

In [8]:
freqs

tensor([ 1.0000, -1.0000,  0.8660, -0.8660,  0.7499, -0.7499,  0.6494,
        -0.6494,  0.5623, -0.5623,  0.4870, -0.4870,  0.4217, -0.4217,
         0.3652, -0.3652,  0.3162, -0.3162,  0.2738, -0.2738,  0.2371,
        -0.2371,  0.2054, -0.2054,  0.1778, -0.1778,  0.1540, -0.1540,
         0.1334, -0.1334,  0.1155, -0.1155,  0.1000, -0.1000,  0.0866,
        -0.0866,  0.0750, -0.0750,  0.0649, -0.0649,  0.0562, -0.0562,
         0.0487, -0.0487,  0.0422, -0.0422,  0.0365, -0.0365,  0.0316,
        -0.0316,  0.0274, -0.0274,  0.0237, -0.0237,  0.0205, -0.0205,
         0.0178, -0.0178,  0.0154, -0.0154,  0.0133, -0.0133,  0.0115,
        -0.0115,  0.0100, -0.0100,  0.0087, -0.0087,  0.0075, -0.0075,
         0.0065, -0.0065,  0.0056, -0.0056,  0.0049, -0.0049,  0.0042,
        -0.0042,  0.0037, -0.0037,  0.0032, -0.0032,  0.0027, -0.0027,
         0.0024, -0.0024,  0.0021, -0.0021,  0.0018, -0.0018,  0.0015,
        -0.0015,  0.0013, -0.0013,  0.0012, -0.0012,  0.0010, -0.0010,
      

In [85]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.l = nn.Linear(2,3)
        self.deop = nn.Dropout(0.1)
    
    def forward(self,x):
        x = self.deop(x)
        return self.l(x)

In [86]:
a = Model()

In [87]:
b = torch.ones(5, 2)
b

tensor([[ 1.,  1.],
        [ 1.,  1.],
        [ 1.,  1.],
        [ 1.,  1.],
        [ 1.,  1.]])

In [110]:
a(b)

tensor([[ 0.3916, -0.5487, -0.4996],
        [ 0.3916, -0.5487, -0.4996],
        [ 0.3916, -0.5487, -0.4996],
        [ 0.3916, -0.5487, -0.4996],
        [ 0.3916, -0.5487, -0.4996]])

In [96]:
a.eval()

Model(
  (l): Linear(in_features=2, out_features=3, bias=True)
  (deop): Dropout(p=0.1)
)